In [1]:
# Import Modules #

In [4]:
import datetime
import pandas as pd
import plotly.offline as plot
import plotly.plotly as py
import plotly.graph_objs as go
plot.offline.init_notebook_mode(connected=True)

In [5]:
## Load Data ##

In [6]:
uuser=pd.read_csv('/home/user/Downloads/ml-100k/u.user',sep = '|',
                  header = None,
                  names = ['user id','age','gender','occupation','zip code'])
udata=pd.read_table('/home/user/Downloads/ml-100k/u.data',
                    header = None,
                    names = ['user id','item id','rating','timestamp'])
uitem = pd.read_csv('/home/user/Downloads/ml-100k/u.item', 
        names = ['item id','movie title','release date','video release date','IMDb URL','unknown','Action',
               'Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film Noir',
               'Horror','Musical','Mystery','Romance','Sci Fi','Thriller','War','Western'],
                sep= '|', header = None, encoding = "ISO-8859-1")

In [5]:
## AGE GROUP HISTOGRAM ##

In [6]:
data = [go.Histogram(x=uuser.age, xbins=dict(start = 0,end = 100,size = 10))]
layout = go.Layout(
    title='FREQUENCY OF AGE',
    xaxis=dict(
        title='AGE'
    ),
    yaxis=dict(
        title='FREQUENCY'
    )
)
fig = go.Figure(data=data, layout=layout)
plot.iplot(fig, filename='histogram')

In [7]:
## OCCUPATION VS AVERAGE RATING ##

In [8]:
merge1=uuser.merge(udata)
merge1.groupby('occupation').mean().rating.index

Index(['administrator', 'artist', 'doctor', 'educator', 'engineer',
       'entertainment', 'executive', 'healthcare', 'homemaker', 'lawyer',
       'librarian', 'marketing', 'none', 'other', 'programmer', 'retired',
       'salesman', 'scientist', 'student', 'technician', 'writer'],
      dtype='object', name='occupation')

In [9]:
trace1 = go.Bar(
    x= list(merge1.groupby('occupation').mean().rating.index),
    y= merge1.groupby('occupation').mean().rating.tolist(),
)
data = [trace1]
layout = dict(
    title='OCCUPATION VS AVERAGE RATING',
    xaxis=dict(
        title='OCCUPATION'
    ),
    yaxis=dict(
        title='AVERAGE RATING'
    ),
    legend=dict(
        traceorder='reversed',
        font=dict(size=16)
    )
)
fig = dict(data=data, layout=layout)
plot.iplot(fig, filename='Occupation vs Average Rating')


In [24]:
## RATINGS HISTOGRAM ##

In [28]:
data = [go.Histogram(x=udata.rating)]
layout = go.Layout(
    title='FREQUENCY OF RATINGS',
    xaxis=dict(
        title='RATINGS'
    ),
    yaxis=dict(
        title='FREQUENCY'
    )
)
fig = go.Figure(data=data, layout=layout)
plot.iplot(fig, filename='histogram')

In [29]:
## AVERAGE RATINGS OVER THE YEARS ##

In [63]:
udata.timestamp=(pd.DatetimeIndex(pd.to_datetime(udata.timestamp, unit='s')).date)

In [69]:
data = [go.Scatter(x=list(udata.groupby('timestamp').mean().rating.index), y=udata.groupby('timestamp').mean().rating.tolist())]

plot.iplot(data)

In [10]:
merge1=udata.merge(uuser)
merge2=udata.merge(uuser)

In [11]:
merge1.drop(['user id','item id','timestamp','occupation','zip code'],axis=1,inplace=True)
merge2.drop(['user id','item id','timestamp','occupation','zip code'],axis=1,inplace=True)

In [12]:
merge1['males']=merge1[merge1.gender=='M'].gender
merge2['females']=merge2[merge2.gender=='F'].gender
merge1.dropna(how='any',inplace=True)
merge2.dropna(how='any',inplace=True)
merge1.drop('gender',axis=1,inplace=True)
merge2.drop('gender',axis=1,inplace=True)

In [13]:
merge2.columns

Index(['rating', 'age', 'females'], dtype='object')

In [14]:
merge1.columns

Index(['rating', 'age', 'males'], dtype='object')

In [15]:
m=merge1.groupby('age').mean().rating
f=merge2.groupby('age').mean().rating
m1=m.to_frame()
f1=f.to_frame()
m1.index.name=None
m1['age']=m1.index
f1.index.name=None
f1['age']=f1.index
m1 = m1.rename(columns={'rating':'mrating'})
f1=f1.rename(columns={'rating':'frating'})
gf=pd.concat([m1,f1],axis=1,ignore_index=False)
gf.drop('age',axis=1,inplace=True)
gf.fillna(value=0,inplace=True)
gf['age']=pd.cut(gf.index,bins=range(0,90,10))
gfplot=gf.groupby('age').mean()
gfplot

,mrating,frating
age,,
"(0, 10]",3.577269,0.000000
"(10, 20]",2.977265,3.109054
"(20, 30]",3.447094,3.432614
"(30, 40]",3.523024,3.664806
"(40, 50]",3.632427,3.540161
"(50, 60]",3.715564,2.975375
"(60, 70]",3.572752,0.722256
"(70, 80]",3.982143,0.000000


In [16]:
trace1 = go.Bar(
    x=gfplot.index,
    y=gfplot.frating,
    name='Female'
)
trace2 = go.Bar(
    x=gfplot.index,
    y=gfplot.mrating,
    name='Male'
)
data = [trace1, trace2]
layout = go.Layout(
    title="Age and Gender vs Average Rating",
    xaxis=dict(
    title='Age and Gender'
    ),
    yaxis=dict(
    range=[0,5],    
    title='Average ratings'
    ),
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
plot.iplot(fig, filename='grouped-bar')


In [17]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
             discriminant_analysis, random_projection)

In [23]:
def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)     
    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(digits.target[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})
    if hasattr(offsetbox, 'AnnotationBbox'):
        ## only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(digits.data.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                ## don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X[i]]]
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
                X[i])
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
t0 = time()
X_tsne = tsne.fit_transform(np.array(X))
plot_embedding(X_tsne,
               "t-SNE embedding of the digits (time %.2fs)" %
               (time() - t0))
plt.show()

Computing t-SNE embedding


MemoryError: 

In [32]:
X = udata.drop(['rating'], axis=1)
Y = udata.rating

In [33]:
np.array(X).shape

(100000, 3)

In [27]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
             discriminant_analysis, random_projection)
## Loading and curating the data
digits = datasets.load_digits(n_class=10)
X = digits.data
y = digits.target

In [29]:
type(X)

numpy.ndarray

In [37]:
type(digits.data)

numpy.ndarray

In [14]:
import numpy as np
from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
#plt.figure(figsize=(10,5))
Y=model.fit_transform(np.array(udata.loc[0:10000,:]))
#plt.scatter(Y[:,0], Y[:,1],np.array(udata1.loc[0:100,'rating']))

#plt.show()

MemoryError: 

In [13]:
trace0 = go.Scatter(
    x = Y[:, 0], 
    y = Y[:, 1],
     mode='markers',
    marker=dict(
        color = udata.loc[0:10000,'rating'],
        showscale=True,
        colorscale='',
    )
)

data = [trace0]

plot.iplot(data)

In [85]:
udata1=pd.read_table('/home/user/Downloads/ml-100k/u5.base',
                    header = None,
                    names = ['user id','item id','rating','timestamp'])

In [52]:
X = (np.array(udata1.drop(['rating'], axis=1)))

In [57]:
 Y = manifold.LocallyLinearEmbedding(10,2,eigen_solver='auto', method='dense').fit_transform(X)

ValueError: unrecognized method 'dense'

In [77]:
Y.shape

(1001, 2)

In [80]:
Y

array([[  5.21717064,   0.68181108],
       [ -4.34362271,  -4.47979065],
       [ 14.22649163,  -3.10085578],
       ..., 
       [  7.25486704, -17.32970829],
       [ -1.17802371,  14.81652082],
       [  2.90291539,  -4.89902621]])

In [7]:
merge1=uuser.merge(udata)
merge2=merge1.merge(uitem)
merge2.columns


Index(['user id', 'age', 'gender', 'occupation', 'zip code', 'item id',
       'rating', 'timestamp', 'movie title', 'release date',
       'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
       'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [8]:
merge2.drop(['user id','zip code','item id','timestamp','movie title','release date','video release date','IMDb URL','unknown'],axis=1,inplace=True)

In [18]:
merge2

,age,gender,occupation,rating,Action,Adventure,Animation,Childrens,Comedy,Crime,...,Fantasy,Film Noir,Horror,Musical,Mystery,Romance,Sci Fi,Thriller,War,Western
0,24,M,technician,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,47,M,educator,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,35,F,other,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,27,M,programmer,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,49,M,educator,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,50,M,healthcare,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,20,M,student,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,26,M,student,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,53,M,programmer,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,25,M,student,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
merge2['occupation']=pd.Categorical(merge2.occupation).codes

In [12]:
import numpy as np
from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
Y=model.fit_transform(np.array(merge2.loc[0:1000,:].drop(['gender','rating'],axis=1)))


In [13]:
trace0 = go.Scatter(
    x = Y[:, 0], 
    y = Y[:, 1],
     mode='markers',
    marker=dict(
        color = merge2.loc[0:1000,'rating'],
        showscale=True,
        colorscale='',
    )
)

data = [trace0]

plot.iplot(data)